In [1]:
import pandas as pd
import numpy as np
import string
import json
import uuid

In [2]:
## Static Config
primary_key = "Identifier"
locations = [
    {
        "column": "Woreda",
        "name": "level1"
    },
    {
        "column": "Kebele",
        "name": "level2"
    }
]


latlong = []
charts = ["odf status"]
default = {
    "odf status" : ["Triggered", "verified","declared"]
}
'''
charts = ["odf status", "initial number latrines", "final number of latrines"]
default = {
    "odf status" : ["Triggered", "declared", "verified"],
    "initial number latrines" : ["initial number latrines"],
    "final number of latrines" : ["final number of latrines"],
}
'''

## Table config
table = []

# Main config
main = {
    'name':'Woreda, Kebele',
    'column': 'Woreda',
    'indicators': [{
        'name':'ID',
        'column': 'Identifier'
    },{
        'name': 'Kebele',
        'column': 'Kebele'
    },{
        'name':'ODF Status',
        'column': 'ODF Status'
    },{
        'name':'Date Triggered',
        'column': 'Date Triggered'
    },{
        'name':'Date of Verification',
        'column': 'Date of Verification'
    },{
        'name':'Progress Time (Days)',
        'column': 'Progress Time'
    },{
        'name':'Time to Complete (Days)',
        'column': 'Time Delta'
    },{
        'name':'Implementing Partner',
        'column': 'Implementing Partner'
    },{
        'name':'Remarks',
        'column': 'Remarks'
    }]
}

In [3]:
df = pd.read_csv('eth_clts_data.csv', error_bad_lines=False)
if 'Identifier' not in list(df):
    df['Identifier'] = [str(uuid.uuid4()) for _ in range(len(df.index))]
    df = df[['Identifier'] + list(df)[0:-1]]

In [4]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [6]:
## Time Delta
def fillTimeDelta(x):
    date_b = pd.to_datetime(x['Date of Verification'])
    if x['Date of Verification'] == '' and x['ODF Status'] == 'Triggered':
        date_b = pd.to_datetime("today")
    return (date_b - pd.to_datetime(x['Date Triggered'])).days

df['Time Delta'] = df.apply(fillTimeDelta, axis=1)
df['Progress Time'] = df.apply(fillTimeDelta, axis=1)

In [7]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [8]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

primary_key_config = ""
chart_config = []
latlong_config = {}

for key in configs:
    if(configs[key].lower() == primary_key.lower()):
        primary_key_config = key
    if(configs[key].lower() in charts):
        chart_config.append({'key': key, 'name': configs[key], 'value': default[configs[key].lower()]})
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})

### Locations
loc_config = {}
for loc in locations:
    column_index = ""
    for key in configs:
        if loc['column'].lower() == configs[key].lower():
            column_index = key
    loc_config.update({loc['name'].lower(): column_index})

#### Table
table_config = []
for tb in table:
    column_index = ""
    tb_indicators = []
    for key in configs:
        if tb['column'].lower() == configs[key].lower():
            column_index = key
        if configs[key].lower() in [x.lower() for x in tb['indicators']]:
            tb_indicators.append(key)
    table_config.append({
        "key": column_index,
        "name": tb['name'],
        "indicators": tb_indicators
    })

## Main Table
main_config = {'name':main['name']}
main_selector = {}
indicators = []
for key in configs:
    if configs[key] == main['column']:
        main_config.update({'key':key})
    for i in main['indicators']:
        if configs[key] == i['column']:
            indicators.append({'name':i['name'],'key':key})
main_config.update({'indicators':indicators})

configs['primary_key'] = primary_key_config
configs['charts'] = chart_config
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config
configs['main'] = main_config

In [9]:
### Replace Dataset Columns
df.columns = index

In [10]:
configs

{'A': 'Identifier',
 'B': 'Woreda',
 'C': 'Kebele',
 'D': 'Village',
 'E': 'Latitude',
 'F': 'Longitude',
 'G': 'No. of HHs',
 'H': 'Initial Number Latrines',
 'I': 'Final Number of Latrines',
 'J': 'Date Triggered',
 'K': 'ODF Status',
 'L': 'Date of Verification',
 'M': 'Implementing Partner',
 'N': 'Remarks',
 'O': 'Time to Complete',
 'P': 'Progress Time',
 'Q': 'Time Delta',
 'primary_key': 'A',
 'charts': [{'key': 'K',
   'name': 'ODF Status',
   'value': ['Triggered', 'verified', 'declared']}],
 'locations': {'level1': 'B', 'level2': 'C'},
 'latlong': {},
 'table': [],
 'main': {'name': 'Woreda, Kebele',
  'key': 'B',
  'indicators': [{'name': 'ID', 'key': 'A'},
   {'name': 'Kebele', 'key': 'C'},
   {'name': 'Date Triggered', 'key': 'J'},
   {'name': 'ODF Status', 'key': 'K'},
   {'name': 'Date of Verification', 'key': 'L'},
   {'name': 'Implementing Partner', 'key': 'M'},
   {'name': 'Remarks', 'key': 'N'},
   {'name': 'Progress Time (Days)', 'key': 'P'},
   {'name': 'Time to C

In [11]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 'adf035a2-a621-4b3c-9948-11f0e053b229',
  'B': 'Arsi Negele',
  'C': 'Ali Wayo',
  'D': 'Zone 1',
  'E': nan,
  'F': nan,
  'G': 400,
  'H': 188,
  'I': nan,
  'J': '22/03/2021',
  'K': 'Triggered',
  'L': '',
  'M': "Amref & local GoV't",
  'N': 'Phase II',
  'O': nan,
  'P': 161,
  'Q': 161},
 {'A': '0ca64527-4fc1-459d-bd66-9ec2affb35de',
  'B': 'Arsi Negele',
  'C': 'Ali Wayo',
  'D': 'Zone 2',
  'E': nan,
  'F': nan,
  'G': 405,
  'H': 190,
  'I': nan,
  'J': '22/03/2021',
  'K': 'Triggered',
  'L': '',
  'M': "Amref & local GoV't",
  'N': 'Phase II',
  'O': nan,
  'P': 161,
  'Q': 161}]

In [12]:
### to json
df.to_json('../resources/js/data/eth_clts_data.json', orient="records", indent=2)

with open('../resources/js/data/eth_clts_config.json', 'w') as outfile:
    json.dump(configs, outfile)